# Using EcoFOCIpy to process raw field data

**Cruise DY2208

**Processed by Shaun Bell**

Follows the initial processing workbook [EcoFOCIpy_sbe_ctd_dy2208l1ipynb](EcoFOCIpy_sbe_ctd_dy2208l1.ipynb) to apply manually corrected csv files to the netcdf files

This will generate:  
+ **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for final preview and validation
- TSSigma, TOXYChlor, TurbParTrans

***TODO:***
+ Apply correction on oxygen as a step function with depth
+ Update any meta data 
+ Apply Oxygen or Salinity Corrections determined after processing

In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xa
import datetime

import EcoFOCIpy.plots.sbe_ctd_plots as sbe_ctd_plots

## Post QC Oxygen / Salinity Processing

In [2]:
#values represent epth ranges
depth_cut = [30000] #endpoint enclusive, #arbitrarily large for no cut
oxy_chan1_multiplier = [0.958,0.958]
oxy_chan2_multiplier = [1.004,1.004]

In [3]:
ncfiles = '.nc'

In [4]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
sample_data_dir = '/Users/bell/ecoraid/2022/CTDcasts/dy2208l2/final_data_cf/ctd/' #root path to cruise directory
cruise_name = 'DY2208L2' #no hyphens
###############################################################

In [5]:
# Following routines will eventually get ported to ecofocipy as subroutines to be called

import seawater as sw

# https://www.oc.nps.edu/nom/day1/parta.html
def sigmat_update(salinity=None,temperature=None):
    '''
    Changes to T or S (commonly to despike values or apply a salinity offset) will need corresponding changes in sigmat
    
        Sigma-t, is density of seawater calculated with in situ salinity
        and temperature, but pressure equal to zero, rather than
        the in situ pressure and 1000 kg/m3 is subtracted.

    '''
    # calculate sigmaT at 0db gauge pressure (s, t, p=0)
    sigt = (sw.eos80.dens0(s=salinity, t=temperature) - 1000)
    
    return sigt


def oxyconc_update(salinity=None,temperature=None, oxygen_conc_umkg=None,pressure=None):
    '''
        Although PJS tends to look at %sat to QC, changes are usually applied on the concentration parameter. So %sat will need recalculation.
        Changes to T/S also drive some small corrections.
        
        Watch the conc units (um/kg or um/l)

        calculate oxygen saturation
        Garcia and Gorden 1992 - from Seabird Derived Parameter Formulas
    '''
    GG_cont = { 'GG_A0':2.00907,
                'GG_A1':3.22014,
                'GG_A2':4.0501,
                'GG_A3':4.94457,
                'GG_A4':-0.256847,
                'GG_A5':3.88767,
                'GG_B0':-0.00624523,
                'GG_B1':-0.00737614,
                'GG_B2':-0.010341,
                'GG_B3':-0.00817083,
                'GG_C0':-0.000000488682}

    Ts_pri = np.log((298.15 - temperature) / (273.15 + temperature))
    Oxsol_pri = np.exp(
    GG_cont['GG_A0']
    + GG_cont['GG_A1'] * Ts_pri
    + GG_cont['GG_A2'] * (Ts_pri) ** 2
    + GG_cont['GG_A3'] * (Ts_pri) ** 3
    + GG_cont['GG_A4'] * (Ts_pri) ** 4
    + GG_cont['GG_A5'] * (Ts_pri) ** 5
    + salinity
    * (GG_cont['GG_B0'] + GG_cont['GG_B1'] * Ts_pri
    + GG_cont['GG_B2'] * (Ts_pri) ** 2 
    + GG_cont['GG_B3'] * (Ts_pri) ** 3)
    + GG_cont['GG_C0'] * (salinity) ** 2
    )

    
    # determine sigmatheta and convert Oxygen from micromoles/kg to ml/l
    # calculate new oxygen saturation percent using derived oxsol
    sigmatheta_pri = sw.eos80.pden(s=salinity, t=temperature, p=pressure)
    oxygen_conc_mll = oxygen_conc_umkg * sigmatheta_pri / 44660
    
    return oxygen_conc_mll,((oxygen_conc_mll) / Oxsol_pri) * 100.0

In [6]:
#match csv to netcdf and update
for cast in sorted(os.listdir(sample_data_dir)):
    if cast.endswith(ncfiles):
        cruise_data_nc = xa.load_dataset(sample_data_dir+cast)
        cruise_data_update = cruise_data_nc.copy()
        try:
            #apply cal correction
            cruise_data_update['oxy_conc_ch1'].values = cruise_data_update['oxy_conc_ch1'].where(cruise_data_update['depth']>depth_cut[0],0) * oxy_chan1_multiplier[1] +\
                                                        cruise_data_update['oxy_conc_ch1'].where(cruise_data_update['depth']<=depth_cut[0],0) * oxy_chan1_multiplier[0]
            cruise_data_update['oxy_conc_ch2'].values = cruise_data_update['oxy_conc_ch2'].where(cruise_data_update['depth']>depth_cut[0],0) * oxy_chan2_multiplier[1] +\
                                                        cruise_data_update['oxy_conc_ch2'].where(cruise_data_update['depth']<=depth_cut[0],0) * oxy_chan2_multiplier[0]
                
            #update 
            #need to update any other oxy conc units too
            cruise_data_update['oxy_concM_ch1'].values,cruise_data_update['oxy_percentsat_ch1'].values = oxyconc_update(cruise_data_update.salinity_ch1,
                                                                   cruise_data_update.temperature_ch1,
                                                                   cruise_data_update.oxy_conc_ch1,
                                                                   cruise_data_update.depth)
            # try:
            cruise_data_update['oxy_concM_ch2'].values, cruise_data_update['oxy_percentsat_ch2'].values = oxyconc_update(cruise_data_update.salinity_ch2,
                                                               cruise_data_update.temperature_ch2,
                                                               cruise_data_update.oxy_conc_ch2,
                                                               cruise_data_update.depth)            
            # except:
            #     pass # no secondary oxy
            
            try:
                cruise_data_update.attrs.update({'history':(cruise_data_update.history + f"Oxy Chan 1 Winkler Slope Correction: {oxy_chan1_multiplier[0]} : < {depth_cut[0]}m "+ str(datetime.datetime.today()) + '\n')})
                cruise_data_update.attrs.update({'history':(cruise_data_update.history + f"Oxy Chan 1 Winkler Slope Correction: {oxy_chan1_multiplier[1]} : >= {depth_cut[0]}m "+ str(datetime.datetime.today()) + '\n')})
                cruise_data_update.attrs.update({'history':(cruise_data_update.history + f"Oxy Chan 2 Winkler Slope Correction: {oxy_chan2_multiplier[0]} : < {depth_cut[0]}m "+ str(datetime.datetime.today()) + '\n')})
                cruise_data_update.attrs.update({'history':(cruise_data_update.history + f"Oxy Chan 2 Winkler Slope Correction: {oxy_chan2_multiplier[1]} : >= {depth_cut[0]}m "+ str(datetime.datetime.today()) + '\n')})
            except: #cause history isn't an attribute yet
                cruise_data_update.attrs['history'] = f"Oxy Chan 1 Winkler Slope Correction: {oxy_chan1_multiplier[0]} : < {depth_cut[0]}m "+ str(datetime.datetime.today()) + '\n'
                cruise_data_update.attrs.update({'history':(cruise_data_update.history + f"Oxy Chan 1 Winkler Slope Correction: {oxy_chan1_multiplier[1]} : >= {depth_cut[0]}m "+ str(datetime.datetime.today()) + '\n')})
                cruise_data_update.attrs.update({'history':(cruise_data_update.history + f"Oxy Chan 2 Winkler Slope Correction: {oxy_chan2_multiplier[0]} : < {depth_cut[0]}m "+ str(datetime.datetime.today()) + '\n')})
                cruise_data_update.attrs.update({'history':(cruise_data_update.history + f"Oxy Chan 2 Winkler Slope Correction: {oxy_chan2_multiplier[1]} : >= {depth_cut[0]}m "+ str(datetime.datetime.today()) + '\n')})
                
            cruise_data_update.to_netcdf(sample_data_dir+cast.replace(ncfiles,'.updated.nc'),format='NETCDF3_CLASSIC',encoding={'time':{'units':'days since 1900-01-01'}})
            print(f'Updated: {cast}')
        except FileNotFoundError:
            print(f'No file to update: {cast}')

/var/folders/g1/6qjvrwsx0vb6ltz_4gv0tmxr0000jm/T/ipykernel_68074/2521839899.py:38: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  cruise_data_update.to_netcdf(sample_data_dir+cast.replace(ncfiles,'.updated.nc'),format='NETCDF3_CLASSIC',encoding={'time':{'units':'days since 1900-01-01'}})
/var/folders/g1/6qjvrwsx0vb6ltz_4gv0tmxr0000jm/T/ipykernel_68074/2521839899.py:38: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  cruise_data_update.to_netcdf(sample_data_

Updated: DY2208L2c017_ctd.nc
Updated: DY2208L2c019_ctd.nc
Updated: DY2208L2c020_ctd.nc
Updated: DY2208L2c021_ctd.nc
Updated: DY2208L2c022_ctd.nc
Updated: DY2208L2c023_ctd.nc
Updated: DY2208L2c024_ctd.nc
Updated: DY2208L2c025_ctd.nc
Updated: DY2208L2c026_ctd.nc


/var/folders/g1/6qjvrwsx0vb6ltz_4gv0tmxr0000jm/T/ipykernel_68074/2521839899.py:38: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  cruise_data_update.to_netcdf(sample_data_dir+cast.replace(ncfiles,'.updated.nc'),format='NETCDF3_CLASSIC',encoding={'time':{'units':'days since 1900-01-01'}})
/var/folders/g1/6qjvrwsx0vb6ltz_4gv0tmxr0000jm/T/ipykernel_68074/2521839899.py:38: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'nanoseconds' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  cruise_data_update.to_netcdf(sample_d

Updated: DY2208L2c027_ctd.nc
Updated: DY2208L2c028_ctd.nc
Updated: DY2208L2c029_ctd.nc
Updated: DY2208L2c030_ctd.nc
Updated: DY2208L2c031_ctd.nc
Updated: DY2208L2c032_ctd.nc
Updated: DY2208L2c033_ctd.nc
Updated: DY2208L2c034_ctd.nc
Updated: DY2208L2c035_ctd.nc
Updated: DY2208L2c036_ctd.nc


/var/folders/g1/6qjvrwsx0vb6ltz_4gv0tmxr0000jm/T/ipykernel_68074/2521839899.py:38: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'nanoseconds' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  cruise_data_update.to_netcdf(sample_data_dir+cast.replace(ncfiles,'.updated.nc'),format='NETCDF3_CLASSIC',encoding={'time':{'units':'days since 1900-01-01'}})


## Generate Plots


### Make General Plots
- 1:1 plots for paired instruments for each cast (tells if a sensor failed)
- TS_Sigmat, Chlor/Par/Turb, Oxy,Temp
- T/S property property plot
- upcast/downcast plt

In [7]:
filepattern = '.nc'

In [8]:
sample_data_dir+cast.replace('.nc','_TempSalSigmaT.png')

'/Users/bell/ecoraid/2022/CTDcasts/dy2208l2/final_data_cf/ctd/DY2208L2c036_ctd_TempSalSigmaT.png'

In [9]:
for cast in sorted(os.listdir(sample_data_dir)):
    if cast.endswith(filepattern):
        cruise_data_nc = xa.load_dataset(sample_data_dir+cast)
        ctd_df = cruise_data_nc.to_dataframe()
        
        #calc sigmat
        ctd_df['sigma_t_ch1'] = sigmat_update(temperature=ctd_df['temperature_ch1'],salinity=ctd_df['salinity_ch1'])
        ctd_df['sigma_t_ch2'] = sigmat_update(temperature=ctd_df['temperature_ch2'],salinity=ctd_df['salinity_ch2'])

        sbe_p = sbe_ctd_plots.CTDProfilePlot(stylesheet='seaborn-v0_8-ticks')
        plt,fig =sbe_p.plot3var(varname=['temperature_ch1','temperature_ch2','salinity_ch1','salinity_ch2','sigma_t_ch1','sigma_t_ch2'],
                          xdata=[ctd_df.temperature_ch1,ctd_df.temperature_ch2,ctd_df.salinity_ch1,ctd_df.salinity_ch2,ctd_df.sigma_t_ch1,ctd_df.sigma_t_ch2],
                          ydata=ctd_df.index.get_level_values('depth'),
                          secondary=True,
                          xlabel=['Temperature','Salinity','SigmaT'])

        DefaultSize = fig.get_size_inches()
        fig.set_size_inches( (DefaultSize[0], DefaultSize[1]*3) )
        plt.savefig(sample_data_dir+cast.replace('.nc','_TempSalSigmaT.png'))
        plt.close(fig)

/Users/bell/src/ecofocipy/src/EcoFOCIpy/plots/sbe_ctd_plots.py:222: RuntimeWarning: All-NaN axis encountered
  abmin=np.nanmin([np.nanmin(xdata[2]),np.nanmin(xdata[3])])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/plots/sbe_ctd_plots.py:223: RuntimeWarning: All-NaN axis encountered
  abmax=np.nanmax([np.nanmax(xdata[2]),np.nanmax(xdata[3])])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/plots/sbe_ctd_plots.py:251: RuntimeWarning: All-NaN axis encountered
  abmin=np.nanmin([np.nanmin(xdata[4]),np.nanmin(xdata[5])])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/plots/sbe_ctd_plots.py:252: RuntimeWarning: All-NaN axis encountered
  abmax=np.nanmax([np.nanmax(xdata[4]),np.nanmax(xdata[5])])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/plots/sbe_ctd_plots.py:222: RuntimeWarning: All-NaN axis encountered
  abmin=np.nanmin([np.nanmin(xdata[2]),np.nanmin(xdata[3])])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/plots/sbe_ctd_plots.py:223: RuntimeWarning: All-NaN axis encountered
  abmax=np.nanmax([np.nanmax(xdata[2]),np

In [10]:
for cast in sorted(os.listdir(sample_data_dir)):
    if cast.endswith(filepattern):
        cruise_data_nc = xa.load_dataset(sample_data_dir+cast)
        ctd_df = cruise_data_nc.to_dataframe()
        
        sbe_p = sbe_ctd_plots.CTDProfilePlot(stylesheet='seaborn-v0_8-ticks')
        plt,fig =sbe_p.plot2var(varname=['temperature_ch1','temperature_ch2','oxy_percentsat_ch1','oxy_percentsat_ch2'],
                          xdata=[ctd_df.temperature_ch1,ctd_df.temperature_ch2,ctd_df.oxy_percentsat_ch1,ctd_df.oxy_percentsat_ch2],
                          ydata=ctd_df.index.get_level_values('depth'),
                          secondary=True,
                          xlabel=['Temperature','Oxygen Saturation'])

        DefaultSize = fig.get_size_inches()
        fig.set_size_inches( (DefaultSize[0], DefaultSize[1]*3) )
        plt.savefig(sample_data_dir+cast.replace('.nc','_TempOxy.png'))
        plt.close(fig)

/Users/bell/src/ecofocipy/src/EcoFOCIpy/plots/sbe_ctd_plots.py:108: RuntimeWarning: All-NaN axis encountered
  abmin=np.nanmin([np.nanmin(xdata[2]),np.nanmin(xdata[3])])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/plots/sbe_ctd_plots.py:109: RuntimeWarning: All-NaN axis encountered
  abmax=np.nanmax([np.nanmax(xdata[2]),np.nanmax(xdata[3])])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/plots/sbe_ctd_plots.py:108: RuntimeWarning: All-NaN axis encountered
  abmin=np.nanmin([np.nanmin(xdata[2]),np.nanmin(xdata[3])])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/plots/sbe_ctd_plots.py:109: RuntimeWarning: All-NaN axis encountered
  abmax=np.nanmax([np.nanmax(xdata[2]),np.nanmax(xdata[3])])


In [11]:
for cast in sorted(os.listdir(sample_data_dir)):
    if cast.endswith(filepattern):
        cruise_data_nc = xa.load_dataset(sample_data_dir+cast)
        ctd_df = cruise_data_nc.to_dataframe()
        
        sbe_p = sbe_ctd_plots.CTDProfilePlot(stylesheet='seaborn-v0_8-ticks')
        plt,fig =sbe_p.plot3var(varname=['turbidity','','chlor_fluorescence','','par',''],
                          xdata=[ctd_df.turbidity,np.array([]),ctd_df.chlor_fluorescence,np.array([]),ctd_df.par,np.array([])],
                          ydata=ctd_df.index.get_level_values('depth'),
                          secondary=False,
                          xlabel=['Turbidity','Fluor','PAR'])

        DefaultSize = fig.get_size_inches()
        fig.set_size_inches( (DefaultSize[0], DefaultSize[1]*3) )
        plt.title(f'Cast:{cast}\nLat:{cruise_data_nc.latitude.values} Lon:{cruise_data_nc.longitude.values}\nTime:{cruise_data_nc.time.values}')
        plt.savefig(sample_data_dir+cast.replace('.nc','_ParTurbFluor.png'))
        plt.close(fig)